<div class="alert alert-block alert-danger">

# FIT5196 Task 2 in Assessment 1
    
#### Student Name: Kehan Liu
#### Student ID: 32281943

Date: 02/09/2022

Environment: Python 3.9.7

Libraries used:
* os (for interacting with the operating system, included in Python xxxx) 
* pandas 1.1.0 (for dataframe, installed and imported) 
* multiprocessing (for performing processes on multi cores, included in Python 3.6.9 package) 
* itertools (for performing operations on iterables)
* nltk 3.5 (Natural Language Toolkit, installed and imported)
* nltk.collocations (for finding bigrams, installed and imported)
* nltk.tokenize (for tokenization, installed and imported)
* nltk.stem (for stemming the tokens, installed and imported)

    </div>

<div class="alert alert-block alert-info">
    
## Table of Contents

</div>

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Input File](#examine) <br>
[4. Loading and Parsing Files](#load) <br>
$\;\;\;\;$[4.1. Tokenization](#tokenize) <br>
$\;\;\;\;$[4.2. Whatever else](#whetev) <br>
$\;\;\;\;$[4.3. Finding First 200 Meaningful Bigrams](#bigrams) <br>
$\;\;\;\;$[4.4. Stemming the word and sorting](#whetev1) <br>
[5. Writing Output Files](#write) <br>
$\;\;\;\;$[5.1. Vocabulary List](#write-vocab) <br>
$\;\;\;\;$[5.2. Sparse Matrix](#write-sparseMat) <br>
[7. References](#Ref) <br>

<div class="alert alert-block alert-success">
    
## 1.  Introduction  <a class="anchor" name="Intro"></a>

This assessment concerns textual data and the aim is to extract data, process them, and transform them into a proper format. The dataset provided is in the format of a PDF file containing the productconverting the extracted data into a numeric representation. ids and product reviews that belongs to the different product id.

<div class="alert alert-block alert-success">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>

In this assessment, any python packages is permitted to be used. The following packages were used to accomplish the related tasks:

* **os:** to interact with the operating system, e.g. navigate through folders to read files
* **re:** to define and use regular expressions
* **pandas:** to work with dataframes
* **nltk:**   A language toolkit that also includes a diverse set of corpora and lexical resources
* **nltk.probability:**  nltk built-in function
* **nltk.collocations:** nltk built-in function for collect multiple words which commonly co-occur.
* **RegexpTokenizer:** splits a string into tokens using a regular expression
* **MWETokenizer:** using for re token the word
* **PorterStemmer:** remove those affixes required for eg. grammatical role, tense, derivational morphology leaving only the stem of the word.
* **math:** provides access to the mathematical functions defined by the C standard.
* **CountVectorizer:** Convert a collection of text documents to a matrix of token counts.


In [1]:
import os
import re
import pandas as pd
from itertools import chain
import nltk
from nltk.probability import *
from nltk.collocations import *
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
from nltk.stem import PorterStemmer
import math
from sklearn.feature_extraction.text import CountVectorizer


-------------------------------------

<div class="alert alert-block alert-success">
    
## 3.  Examining Input File <a class="anchor" name="examine"></a>

Let's examine what is the content of the file. For this purpose, we need to transfer the PDF file to txt file while using pdfminer.

In [1]:
!pdf2txt.py -o 32281943_task2.txt /Users/haohaoxuexi/FIT5196/ass1/32281943_task2.pdf
print('done')

done


In [5]:
with open('32281943_task2.txt','r') as f1:   
    content = f1.readlines()
content

['B00004TJ7O\n',
 '\n',
 "['What a workhorse, we bought the Oly C-3000z around the end of 2000 and, until very recently,\n",
 'have used it at least a few times a week (almost every week) in our shop for product pictures. With a\n',
 'resolution of 3.2mp, this camera takes pictures that are still good for web pages and most\n',
 'advertising flyers and prints we produce.It might seem odd to be writing a review for a camera this\n',
 'old, but we are about to retire this one from the shop (replacing it withFujifilm Finepix F70EXR 10MP\n',
 'Super CCD Digital Camera with 10x Optical Triple Image Stabilized Zoom and 2.7 inch LCD) and\n',
 'give it to one of my kids as a beater camera. I noticed that several of these cameras were being sold\n',
 'on Amazon as used (but not by me), and wanted to let people know that it is possible for there to be\n',
 '"life" in a camera this old.Here are my general observations (+/-):+ Good Lens - 3x Optical Zoom-\n',
 'Shows "Purple Fringing" in some case

It is noteiced that file contains the product id and the product review. 

<div class="alert alert-block alert-success">
    
## 4.  Loading and Parsing File <a class="anchor" name="load"></a>

In this section, we using regex to extract the product id and product review, then store them to a dictionary since the product id and product review is one to one correspondence.

In [6]:
dict_content = {}
tokenized_dict = {}
with open('32281943_task2.txt','r') as f1:   
    content = f1.read()
    pid = re.findall(r'(B00\S{7}|\d{9}\S)(?:\s\s)', content)
    review = re.findall(r'(?:\[)(.*(?:\n.*)*?\n.*)(?:\]\n\n)', content)  
    dict_content = dict(zip(pid, review))

In [7]:
len(pid)

100

In [8]:
len(review)

100

The lenth of product id and product review are same, so the data extract is correct

Let's examine the dictionary generated. 

In [10]:
dict_content['B00004TJ7O']

'\'What a workhorse, we bought the Oly C-3000z around the end of 2000 and, until very recently,\nhave used it at least a few times a week (almost every week) in our shop for product pictures. With a\nresolution of 3.2mp, this camera takes pictures that are still good for web pages and most\nadvertising flyers and prints we produce.It might seem odd to be writing a review for a camera this\nold, but we are about to retire this one from the shop (replacing it withFujifilm Finepix F70EXR 10MP\nSuper CCD Digital Camera with 10x Optical Triple Image Stabilized Zoom and 2.7 inch LCD) and\ngive it to one of my kids as a beater camera. I noticed that several of these cameras were being sold\non Amazon as used (but not by me), and wanted to let people know that it is possible for there to be\n"life" in a camera this old.Here are my general observations (+/-):+ Good Lens - 3x Optical Zoom-\nShows "Purple Fringing" in some cases+ Good Solid Body - Feels good in the hand- Bad Lens Cap\n- Replaced 

 We can see that the key of dictionary is the product id and the value now is the review of belongs to this product id.
 

<div class="alert alert-block alert-warning">
    
### 4.1. Tokenization <a class="anchor" name="tokenize"></a>

Tokenization is a principal step in text processing and producing unigrams and bigrams. In this section, we use the RegexpTokenizer function to generate the token while using regular expressions

In [12]:
tokenizer = RegexpTokenizer("[a-zA-Z]+(?:[-'][a-zA-Z]+)?")
for key, value in dict_content.items():
    tokenized_dict[key] = tokenizer.tokenize(value.lower())

The above operation results in a dictionary with PID representing keys and a single string for all reviews of the day concatenated to each other. ...

In [13]:
tokenized_dict

{'B00004TJ7O': ['what',
  'a',
  'workhorse',
  'we',
  'bought',
  'the',
  'oly',
  'c',
  'z',
  'around',
  'the',
  'end',
  'of',
  'and',
  'until',
  'very',
  'recently',
  'have',
  'used',
  'it',
  'at',
  'least',
  'a',
  'few',
  'times',
  'a',
  'week',
  'almost',
  'every',
  'week',
  'in',
  'our',
  'shop',
  'for',
  'product',
  'pictures',
  'with',
  'a',
  'resolution',
  'of',
  'mp',
  'this',
  'camera',
  'takes',
  'pictures',
  'that',
  'are',
  'still',
  'good',
  'for',
  'web',
  'pages',
  'and',
  'most',
  'advertising',
  'flyers',
  'and',
  'prints',
  'we',
  'produce',
  'it',
  'might',
  'seem',
  'odd',
  'to',
  'be',
  'writing',
  'a',
  'review',
  'for',
  'a',
  'camera',
  'this',
  'old',
  'but',
  'we',
  'are',
  'about',
  'to',
  'retire',
  'this',
  'one',
  'from',
  'the',
  'shop',
  'replacing',
  'it',
  'withfujifilm',
  'finepix',
  'f',
  'exr',
  'mp',
  'super',
  'ccd',
  'digital',
  'camera',
  'with',
  'x',


At this stage, all reviews for each PID are tokenized and are stored as a value in the new dictionary (separetely for each day).

-------------------------------------

<div class="alert alert-block alert-warning">
    
### 4.2. Finding First 200 Meaningful Bigrams <a class="anchor" name="bigrams"></a>

One of the tasks is to find the first 200 meaningful bigrams. These bigrams should also be included in the final vocabulary list. The first step is to concatenate all the tokenized patents using the chain.frome_iterable function. The returned list by the function contains a list of all the words seprated by while space.

In [14]:
word = list(chain.from_iterable(tokenized_dict.values()))

Then, find first 200 meaningful bigrams while using PMI measure

In [15]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(word)
first_200_bigram = bigram_finder.nbest(bigram_measures.pmi,200)

After find the first 200 bigram, using list to store these bigrams and change them format like sample vocab output.

In [16]:
first_200_bigram_list = [each[0] + '_' + each[1] for each in first_200_bigram]

Having found the top 200 meaningful bigrams, we need to retokenize tweets considering the bigrams as well.....

In [17]:
re_token_bigrams_dict = {}
re_token = MWETokenizer(first_200_bigram)
for key, value in tokenized_dict.items():
    re_token_bigrams_dict[key] = re_token.tokenize(value)

At this stage, we have a dictionary of tokenized words, whose keys are indicative of PID of reviews. The list of tokens also include the top 200 meaningful bigrams achieved using PMI measure.

-------------------------------------

<div class="alert alert-block alert-warning">
    
### 4.3. Filter the word in stop word list and drop same token <a class="anchor" name="bigrams"></a>

In [19]:
stopwords_list = []
with open('stopwords_en.txt') as stopwords_file:
    stopwords_list = stopwords_file.read().splitlines()

At this stage, transfer the stop word file to a list for filter the words in next part

In [22]:
new_bigram_dict = {}
drop_same_tokens = {}
new_bigram_dict = {key:[word for word in value if len(word)>=3 and word not in stopwords_list]
                   for key, value in re_token_bigrams_dict.items()}
drop_same_tokens = {key: list(set(value)) for key, value in new_bigram_dict.items()}

first create a dictionary with the pid as key, and the value now are the token that length > 3 and not in stopword_list.

After that, create another dictionary with the pid as key, and the value are the token that filter the same words while using set function

At this stage, we have a dictionary of tokenized words that without stop words and duplicate words.

<div class="alert alert-block alert-warning">
    
### 4.3. Filter the context dependent word and rare token  <a class="anchor" name="bigrams"></a>

The first step is to concatenate all word in drop_same_tokens values while using the chain.frome_iterable function. The returned list by the function contains a list of all the words seprated by while space.

In [23]:
word_tokens = list(chain.from_iterable(drop_same_tokens.values()))

Then, using FreqDist() function to generate the word frequncy for filter the context dependent word

In [24]:
count_frequncy = FreqDist(word_tokens)

After that, set the threshold for filter the context dependent word. Based on the specification, the thereshold is 
ceil(len(pid)/2)

In [26]:
threshold = math.ceil(len(tokenized_dict.keys()) / 2)

create the list for context dependent filter. Key is the word and value is the word occurance frequency. since the first 200 bigram need to contain in the output, so set the condition that do not filter the word in first_200_bigram_list. Using set function to drop the duplicate words.

In [27]:
context_depend_filter = list(set([key for key, value in count_frequncy.items() if value >= threshold and key not in
                             first_200_bigram_list]))

In [29]:
context_depend_filter

['easy',
 'good',
 'bad',
 'enjoyed',
 'job',
 'part',
 'review',
 'times',
 'found',
 'long',
 'worth',
 'buy',
 'made',
 'time',
 'bit',
 'book',
 'years',
 'wanted',
 'thing',
 'find',
 'give',
 'things',
 'story',
 'stars',
 'people',
 'great',
 'feel',
 'pretty',
 'excellent',
 'set',
 'love',
 'lot',
 'read',
 'recommend',
 'work',
 'nice',
 'end',
 'bought',
 'highly',
 'put',
 'makes',
 'back',
 'make',
 'hard']

Then, create a list for the rare token. Using set function to drop the duplicate word and use list function to tranfer
them to list. select the rare token with the condition that the word frequency less than 10 and the word is not in first_200_bigram_list

In [28]:
rare_token_filter = list(set([key for key, value in count_frequncy.items() if value < 10 and key not in 
                         first_200_bigram_list]))

In [30]:
rare_token_filter

['flourecent',
 'rajan',
 'anther',
 'tort',
 'verbally',
 "someone's",
 'minnie',
 'ortonwhat',
 'underpinnings',
 'unclear',
 'introduction',
 'benefit',
 'comeback',
 'remarked',
 'backwards',
 'rending',
 'calloused',
 "must've",
 'esteemed',
 'warn',
 'haunted',
 "arabella's",
 'obtained',
 'rey',
 'flatten',
 'monster',
 "you's",
 'layered',
 'weinsteins',
 'wyatt',
 'embraces',
 'rational',
 'breathe',
 'requirements',
 'real-time',
 'arises',
 'encounter',
 'trask',
 'perspectives',
 'possess',
 'sucker',
 'charteris',
 'stones',
 'nightspot',
 'warehouse',
 'aspected',
 'spiders',
 'youanyway',
 'christmases',
 'victory',
 'timing',
 'nc-based',
 'weakest',
 'verbal',
 'blank',
 'thighs',
 'rma',
 'compulsively',
 'karaoke-style',
 'estimate',
 'paled',
 'harsh',
 'south',
 'cardinal',
 'killings',
 'percentage',
 'reverts',
 'discovered',
 'stereoscopic',
 'booting',
 'cookie-cutter',
 'dry',
 'sequences',
 'noticeable',
 'ungainly',
 'synthetic',
 'excessively',
 'overtime',

At this stage, we can get a list that without all filter condition. Using the list conprehension to iterate the item in word_token list. Saving the words that not in context_depend_filter and rare_token_filter. Using set function to drop the duplicate words.

In [32]:
vocab_output = list(set([item for item in word_tokens if item not in context_depend_filter and item 
                        not in rare_token_filter]))

<div class="alert alert-block alert-warning">
    
### 4.4. Stemming the word and sorting  <a class="anchor" name="bigrams"></a>

At this stage, we will use the PorterStemmer function to stem the word. Since there have the condition that the word length is less than three after stemming, to consist with the requirement, set the condition to drop the word with length less than 3 while using list comprehension.

In [33]:
stemmer = PorterStemmer()
final_output = []
for each in vocab_output:
    final_output.append(stemmer.stem(each))
final_output = list(set([item for item in final_output if len(item)>=3]))
final_output.sort()

At this stage, we can get the list that sorted alphabetically.

<div class="alert alert-block alert-success">
    
## 5. Writing Output Files <a class="anchor" name="write"></a>

Four files need to be generated:
* Vocabulary list
* Sparse matrix

This is performed in the following sections.

<div class="alert alert-block alert-warning">
    
### 5.1. Vocabulary List <a class="anchor" name="write-vocab"></a>

List of vocabulary should also be written to a file, sorted alphabetically, with their reference codes in front of them. This file also refers to the sparse matrix in the next file. For this purpose, create a dictionary with the item in final_output list as key, and the value is the the index of the word.

In [35]:
output = {}
for ind in range(len(final_output)):
    output[final_output[ind]] = ind 
    
with open('32281943_vocab.txt', 'w') as write_f:
    for key, value in output.items():
        write_f.write(key + ':' + str(value) + '\n')

At this stage, we can get the final vocab file that consist with requirement.

<div class="alert alert-block alert-warning">
    
### 5.2. Sparse Matrix <a class="anchor" name="write-sparseMat"></a>

For writing sparse matrix for each PID, we firstly calculate the frequency of words for that PID.

Since we need to calculate the word frequency which means we need to use the dictionary without drop the duplicate words. So we use the new_bigram_dict for the next steps.

We first filter the word are context dependent and rare token, and stem the words after filter the condition.

Then, get the index of the output word and create the dictionary with the pid as the key and value now is the word index in vocab file.

After that, using the CountVectorizer function to create the sparse matrix.



In [ ]:
with open('32281943_countVec.txt','w') as f1:
    for key, value in new_bigram_dict.items():
        f1.write(key + ',' )
        process_dict = {key:[stemmer.stem(item) for item in value 
                             if item not in context_depend_filter and item not in rare_token_filter]}
        vector_dict = {k:[output[each] for each in v if len(each)>=3] for k, v in process_dict.items()}
        for pid, ind in vector_dict.items():
            vectornizer = CountVectorizer(analyzer='word', ngram_range = (1,2))
            ind = [str(each) for each in ind]
            features = vectornizer.fit_transform([' '.join(ind)])
            feature_name = vectornizer.get_feature_names() 
            for key, value in zip(feature_name, features.toarray()[0]):
                if key == feature_name[-1]:
                    f1.write(key + ':' + str(value))
                else:
                     f1.write(key + ':' + str(value) + ',')
        f1.write('\n')

At this stage, we will get the sparse matrix file with the format like sample.

-------------------------------------

<div class="alert alert-block alert-success">
    
## 6. References <a class="anchor" name="Ref"></a>

[1] sklearn.feature_extraction.text.CountVectorizer , https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html, Accessed 30/08/2022.

[2] NLTK Documentation-Collocations , https://www.nltk.org/howto/collocations.html, Accessed 30/08/2022.

[3] NLTK Documentation-nltk.stem package , https://www.nltk.org/api/nltk.stem.html, Accessed 30/08/2022.

[4]math — Mathematical functions , https://docs.python.org/3/library/math.html, Accessed 30/08/2022.




## --------------------------------------------------------------------------------------------------------------------------